In [4]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import scipy
import psycopg2
import tensorflow as ts
from collections import defaultdict

con = psycopg2.connect(database='codeforces', user='Joy')
cur = con.cursor()

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 10.0)
plt.rcParams['figure.facecolor'] = 'white'

In [5]:
from sqlalchemy import create_engine
engine = create_engine('postgres://%s@localhost/%s'%('Joy', 'codeforces'))

# create Y values

In [6]:
# note this is 4x faster than getting it from sql
df_smooth = pd.read_csv('user_ratings_smoothed.csv', engine = 'c')

## calculate difference
Only need to run this once
```
gusr = df_smooth.groupby('handle')
stack = []

for usr, dfu in gusr:
    dfu.is_copy=False
    dfu.sort_values('ratingupdatetimeseconds', inplace=True)
    stack.append(dfu)

df_smooth = pd.concat(stack)
for month in range(1, 6):
    curr = df_smooth["smoothed_%dmonths" % month]
    prev = np.roll(curr, 1)

    delta = curr - prev
    df_smooth["delta_smoothed_%dmonths" % month] = delta

df_smooth.head(50)

## output to sql and csv

df_smooth.to_csv('user_ratings_smoothed.csv', index=False, header=True)

df_smooth.to_sql('user_rating_smooth', engine, if_exists='replace')
```

# Features
 **problem type**
 * contest
 * virtual
 * etc
 
**problem info**
 * tags
 * rating
 * point value
 
**submission info**
 * number of wrong answers
 * number of TLE
 * number of compile errors
 * time between first submission and solve
 * relative time to competition
 
**user info**
 * current smooth rating
 * volatility?
 * lag can be estimated from user rating and smoothed rating, but do we want it??

## Getting distinct values for categorial variables

In [7]:
cur.execute("SELECT * FROM handles")
all_handles = [h[0] for h in cur.fetchall()]

Run this code **once only** to get a list of keys. This takes ~20 minutes

```
q = """
SELECT DISTINCT handle, contestid, problemid FROM submissions;
"""
cur.execute(q)
keys = cur.fetchall()

with open('handle_cid_pid_keys.txt', 'w') as f:
    for k in keys:
        f.write(','.join(k) + '\n')
```

In [8]:
with open('handle_cid_pid_keys.txt') as f:
    keys = [line.strip() for line in f.readlines()]

### Get all tags
Only need to run this once:
```
cur.execute("""
SELECT DISTINCT tag FROM tags
""")
all_tags = [t[0] for t in cur.fetchall()]

df_all_tags = pd.DataFrame(all_tags)
df_all_tags.rename_axis({0: 'tag'}, axis=1, inplace=True)
df_all_tags.to_sql('all_tags', engine, if_exists='replace')
```

In [9]:
cur.execute("SELECT tag FROM all_tags")
all_tags = set([t[0] for t in cur.fetchall()])

### Get distinct verdicts

Only need to run this once:
```
cur.execute("""
SELECT DISTINCT verdict FROM submissions
""")
all_verdicts = [v[0] for v in cur.fetchall()]

df_all_verdicts = pd.DataFrame(all_verdicts)
df_all_verdicts.rename_axis({0: 'verdict'}, axis=1, inplace=True)
df_all_verdicts.to_sql('all_verdicts', engine, if_exists='replace')
```

In [10]:
cur.execute("SELECT verdict FROM all_verdicts")
all_verdicts = set([t[0] for t in cur.fetchall()])

### Get distinct participant types
```
cur.execute("""
SELECT DISTINCT participanttype FROM submissions
""")
all_participanttypes = [v[0] for v in cur.fetchall()]

df_all_participanttypes = pd.DataFrame(all_participanttypes)
df_all_participanttypes.rename_axis({0: 'participanttype'}, axis=1, inplace=True)
df_all_participanttypes.to_sql('all_participanttypes', engine, if_exists='replace')
```

In [11]:
cur.execute("SELECT participanttype FROM all_participanttypes")
all_participanttypes = set([t[0] for t in cur.fetchall()])

### programming languages
```
cur.execute("""
SELECT DISTINCT language FROM submissions
""")
all_language = [v[0] for v in cur.fetchall()]

df_all_language = pd.DataFrame(all_language)
df_all_language.rename_axis({0: 'language'}, axis=1, inplace=True)
df_all_language.to_sql('all_language', engine, if_exists='replace')
```

In [16]:
cur.execute("SELECT language FROM all_language")
all_language = set([t[0] for t in cur.fetchall()])

## problem stats

### problem rating and tags

In [12]:
df_prate = pd.read_sql("SELECT * FROM problem_rating", con)
df_prate.set_index(['contestid', 'problemid'], inplace=True)

df_tags = pd.read_sql("SELECT * FROM tags", con)
df_tags.set_index(['contestid', 'problemid'], inplace=True)

df_smooth.reset_index(inplace=True)
df_smooth.set_index(['handle'], inplace=True)
df_smooth.drop('contestname', axis=1, inplace=True)
df_smooth.drop('time', axis=1, inplace=True)

In [13]:
user_dict = defaultdict(list)
keys = [k.split(',') for k in keys]
for k in keys:
    user_dict[ k[0] ].append(k)

In [61]:
present_handles = set(df_smooth.index)

In [54]:
from os.path import exists
cnt = 0
#user = '-----'

def getTraining(user):
    #filename = 'train_rnn.csv'
    filename = 'rnn_train/%s.csv' % user
    print filename
    trainlist = []
    ur = df_smooth.loc[user, :]
    if len(ur.shape) == 1:
        print "     Not enough contests for user", ur.shape
        return
    #ur.drop(['contestid', 'rank'], inplace=True, axis=1)
    ur.is_copy = False
    ur.reset_index(inplace=True)
    
    
    for k in user_dict[user]:
        q = """
        SELECT * FROM submissions
            WHERE
                handle = '%s'
                AND
                contestid = '%s'
                AND
                problemid = '%s'
        """ % (k[0], k[1], k[2])

        df = pd.read_sql(q, con)
        df.is_copy = False

        ex = dict()
        
        # generic problem info
        ex['points'] = df.points[0]
        ex['starttimeseconds'] = min(df.starttimeseconds)
        ex['stoptimeseconds'] = max(df.starttimeseconds)
        
        # user rating info ----------------------------------
        # find closest next contest
        # if there is no next contest,then skip this entry
        idx = ur.ratingupdatetimeseconds >= ex['stoptimeseconds']
        if not np.any(idx):
            continue
        tur = ur.loc[idx]
        tur.is_copy = False
        idx = tur.ratingupdatetimeseconds == min(tur.ratingupdatetimeseconds)
        tur = tur.loc[idx].to_dict(orient='records')[0]
        ex.update(tur)


        # verdicts
        vcnt = df.verdict.value_counts()
        vdict = vcnt.to_dict()
        ex.update(vdict)

        # participant type
        pcnt = df.participanttype.value_counts()
        pdict = pcnt.to_dict()
        for t in pdict.iterkeys():
            ex[t] = 1

        # language
        lcnt = df.language.value_counts()
        ldict = lcnt.to_dict()
        ex.update(ldict)

        # problem rating
        if (k[1], k[2]) in df_prate:
            ex['problem_rating'] = df_prate.loc[k[1],k[2]].values[0]
        else:
            ex['problem_rating'] = -1

        # time to solves
        solvetime = df.loc[df.verdict=='OK', 'starttimeseconds']
        if len(solvetime) > 0:
            ex['solvetimeseconds'] = min(solvetime)
        else:
            ex['solvetimeseconds'] = -1

        trainlist.append(ex)

    #    cnt += 1
    #    if cnt > 10:
    #        break

    df_train = pd.DataFrame.from_dict(trainlist)
    for t in all_tags:
        if t not in df_train.columns:
            df_train[t] = np.nan
    for t in all_verdicts:
        if t not in df_train.columns:
            df_train[t] = np.nan
    for t in all_participanttypes:
        if t not in df_train.columns:
            df_train[t] = np.nan
    for t in all_language:
        if t not in df_train.columns:
            df_train[t] = np.nan
            
#    if df_train.shape[1] != 100:
#        print df_train.columns

#    if exists(filename):
    df_train.to_csv(filename, mode='w', index=False, header=True)
#    else:
#        df_train.to_csv(filename, mode='a', index=False, header=True)

In [34]:
all_handles[5012]

'vikky_codder'

In [63]:
lastidx = 22529
for i, user in enumerate(all_handles[lastidx:]):
    if user in present_handles:
        print lastidx + i, user
        getTraining(user)

22529 huatian
rnn_train/huatian.csv
22530 hzf20115455
rnn_train/hzf20115455.csv
22532 johny05
rnn_train/johny05.csv
22534 kagamine_len
rnn_train/kagamine_len.csv
22535 kond
rnn_train/kond.csv
22537 kritishah
rnn_train/kritishah.csv
22538 lavina_jain
rnn_train/lavina_jain.csv
22539 leecopper15
rnn_train/leecopper15.csv
22540 leisurer
rnn_train/leisurer.csv
22541 lraszkiewicz
rnn_train/lraszkiewicz.csv
22542 luiz
rnn_train/luiz.csv
22543 lyx613
rnn_train/lyx613.csv
22544 m.samir
rnn_train/m.samir.csv
22546 nahian_002
rnn_train/nahian_002.csv
22550 orailly
rnn_train/orailly.csv
22552 perfect
rnn_train/perfect.csv
22553 projectstowork1
rnn_train/projectstowork1.csv
22554 protinocode
rnn_train/protinocode.csv
22556 sanchitkum
rnn_train/sanchitkum.csv
22557 satyam
rnn_train/satyam.csv
22558 sdu.yh0104
rnn_train/sdu.yh0104.csv
22559 sentinel45
rnn_train/sentinel45.csv
22560 shad
rnn_train/shad.csv
22562 sndtkrh
rnn_train/sndtkrh.csv
22564 sumeetiiita
rnn_train/sumeetiiita.csv
22565 sumit99531

22827 Javanochka
rnn_train/Javanochka.csv
22828 JerryCho
rnn_train/JerryCho.csv
22829 Julian.
rnn_train/Julian..csv
22832 Miha
rnn_train/Miha.csv
22833 NickAltair
rnn_train/NickAltair.csv
22834 Nigel
rnn_train/Nigel.csv
22836 Priyanshu_Kumar
rnn_train/Priyanshu_Kumar.csv
22837 QyQ
rnn_train/QyQ.csv
22840 TxCowboy
rnn_train/TxCowboy.csv
22841 _HYX_
rnn_train/_HYX_.csv
22842 archer01
rnn_train/archer01.csv
     Not enough contests for user (16,)
22843 arnavkj95
rnn_train/arnavkj95.csv
22844 b_r
rnn_train/b_r.csv
22845 bondanvak
rnn_train/bondanvak.csv
22847 cholerikasi
rnn_train/cholerikasi.csv
22849 cpy
rnn_train/cpy.csv
22850 cxw
rnn_train/cxw.csv
22851 enitihas
rnn_train/enitihas.csv
22852 fffgrdcc
rnn_train/fffgrdcc.csv
22856 holc
rnn_train/holc.csv
22857 hujiaxi
rnn_train/hujiaxi.csv
22859 its_always_BATMAN
rnn_train/its_always_BATMAN.csv
22860 jiejie
rnn_train/jiejie.csv
22861 jwl
rnn_train/jwl.csv
22862 khandeshb1
rnn_train/khandeshb1.csv
22863 liuyubobobo
rnn_train/liuyubobobo.cs

23136 KareemMohsen
rnn_train/KareemMohsen.csv
23137 KiRiN
rnn_train/KiRiN.csv
23139 MessyShen
rnn_train/MessyShen.csv
23140 Pajaraja
rnn_train/Pajaraja.csv
23142 ReginaL
rnn_train/ReginaL.csv
23143 RoyalWyvern
rnn_train/RoyalWyvern.csv
23144 Tanim_kuet
rnn_train/Tanim_kuet.csv
23145 Wahshi
rnn_train/Wahshi.csv
23148 acherepkov
rnn_train/acherepkov.csv
23149 aidarbiktimirov
rnn_train/aidarbiktimirov.csv
23150 alexlopashev
rnn_train/alexlopashev.csv
23151 artefom
rnn_train/artefom.csv
23153 aurevoir
rnn_train/aurevoir.csv
23155 cympfh
rnn_train/cympfh.csv
23157 dathvan
rnn_train/dathvan.csv
     Not enough contests for user (16,)
23158 derxyz
rnn_train/derxyz.csv
23159 devils13
rnn_train/devils13.csv
23160 dimamol
rnn_train/dimamol.csv
23162 donatello
rnn_train/donatello.csv
23166 tada-s
rnn_train/tada-s.csv
23167 gaurav1997garg
rnn_train/gaurav1997garg.csv
23168 glebrubinski
rnn_train/glebrubinski.csv
23169 hAtchiko
rnn_train/hAtchiko.csv
23170 harmjan
rnn_train/harmjan.csv
23172 hermit

23436 savask
rnn_train/savask.csv
23437 schpnc
rnn_train/schpnc.csv
23438 sergkolin
rnn_train/sergkolin.csv
23439 sherifmowafy_
rnn_train/sherifmowafy_.csv
23441 shubhi
rnn_train/shubhi.csv
23442 songyiqun
rnn_train/songyiqun.csv
23443 superrabbit
rnn_train/superrabbit.csv
23444 txy
rnn_train/txy.csv
23445 vivekjnv93
rnn_train/vivekjnv93.csv
23447 xiaohaoK1
rnn_train/xiaohaoK1.csv
23449 zgmf_x20a
rnn_train/zgmf_x20a.csv
23453 Amir169
rnn_train/Amir169.csv
23454 Animax13
rnn_train/Animax13.csv
23455 AymanWizard
rnn_train/AymanWizard.csv
23456 BersinSepatu
rnn_train/BersinSepatu.csv
23457 CZJ
rnn_train/CZJ.csv
23459 ErizoSteep
rnn_train/ErizoSteep.csv
23462 Godotcoffee
rnn_train/Godotcoffee.csv
23464 I_love_Ha_Phuong
rnn_train/I_love_Ha_Phuong.csv
23465 Imangali_Temirhan
rnn_train/Imangali_Temirhan.csv
23466 IslamSalah
rnn_train/IslamSalah.csv
23467 Just_ck
rnn_train/Just_ck.csv
23468 LNPED
rnn_train/LNPED.csv
23469 LuaXD
rnn_train/LuaXD.csv
23470 LuckyLQD
rnn_train/LuckyLQD.csv
23472 Pi

23738 TrickyG
rnn_train/TrickyG.csv
23739 WizardMagius
rnn_train/WizardMagius.csv
23740 XaCaHaa
rnn_train/XaCaHaa.csv
23741 aayush9
rnn_train/aayush9.csv
23742 aivyulik
rnn_train/aivyulik.csv
23743 al__gol
rnn_train/al__gol.csv
23744 aya_elymany
rnn_train/aya_elymany.csv
23745 bhadreswar10
rnn_train/bhadreswar10.csv
23746 dd.
rnn_train/dd..csv
23747 dujvet
rnn_train/dujvet.csv
     Not enough contests for user (16,)
23749 gaiasabre11
rnn_train/gaiasabre11.csv
23752 hjj
rnn_train/hjj.csv
23754 i.nagata110
rnn_train/i.nagata110.csv
23755 jindalshivam09
rnn_train/jindalshivam09.csv
23756 johnfn
rnn_train/johnfn.csv
23757 legusta
rnn_train/legusta.csv
23759 mbaros
rnn_train/mbaros.csv
23761 nextGoalBecomingViolet
rnn_train/nextGoalBecomingViolet.csv
23762 nikiror
rnn_train/nikiror.csv
23763 patina27
rnn_train/patina27.csv
23764 prasodani
rnn_train/prasodani.csv
23765 qwertyasdfgh
rnn_train/qwertyasdfgh.csv
23766 radudoros
rnn_train/radudoros.csv
23767 rasto93
rnn_train/rasto93.csv
23768 sg

...

102626 EVTihonov
rnn_train/EVTihonov.csv
102627 Kenari
rnn_train/Kenari.csv
102628 Shuvo07
rnn_train/Shuvo07.csv
102629 ariefianto17
rnn_train/ariefianto17.csv
102630 black_bird
rnn_train/black_bird.csv
102631 failfailure
rnn_train/failfailure.csv
102632 prantacse14
rnn_train/prantacse14.csv
102633 rkm_coder
rnn_train/rkm_coder.csv
102634 shakil_takeoff
rnn_train/shakil_takeoff.csv
102636 tanmay_jain
rnn_train/tanmay_jain.csv
102637 krchndn
rnn_train/krchndn.csv
102638 partha_cuet
rnn_train/partha_cuet.csv
102641 scorchSupernova
rnn_train/scorchSupernova.csv
102642 tashik
rnn_train/tashik.csv
102643 tuudug
rnn_train/tuudug.csv
102644 Default.
rnn_train/Default..csv
102645 NAHID_39
rnn_train/NAHID_39.csv
102646 Raihanur_R
rnn_train/Raihanur_R.csv
102647 Reigns123
rnn_train/Reigns123.csv
102648 Tolstolutskih
rnn_train/Tolstolutskih.csv
102649 _Kirin_
rnn_train/_Kirin_.csv
102650 abkarinomada
rnn_train/abkarinomada.csv
102651 davit2003
rnn_train/davit2003.csv
102652 karthick_siva
rnn_trai

102853 pias
rnn_train/pias.csv
102854 Nivanjeet
rnn_train/Nivanjeet.csv
102855 RIZATI
rnn_train/RIZATI.csv
102856 fab96
rnn_train/fab96.csv
102857 Deep_13
rnn_train/Deep_13.csv
102858 Hassan52
rnn_train/Hassan52.csv
102859 MOHMMAD_HMEAD
rnn_train/MOHMMAD_HMEAD.csv
102860 Mujakkir00
rnn_train/Mujakkir00.csv
102861 adipu
rnn_train/adipu.csv
102862 asik_b8
rnn_train/asik_b8.csv
102863 hitman47
rnn_train/hitman47.csv
102866 sazedul
rnn_train/sazedul.csv
102867 3.14159265358
rnn_train/3.14159265358.csv
102868 HHMoon
rnn_train/HHMoon.csv
102871 Raarujeed
rnn_train/Raarujeed.csv
102873 Turquoise_Tanz
rnn_train/Turquoise_Tanz.csv
102874 harryp
rnn_train/harryp.csv
102875 kuruma
rnn_train/kuruma.csv
102876 mostafayehya
rnn_train/mostafayehya.csv
102877 Ashik53
rnn_train/Ashik53.csv
102878 Atik_Ahamed
rnn_train/Atik_Ahamed.csv
102879 SHAFIQ_49
rnn_train/SHAFIQ_49.csv
102881 dream2go
rnn_train/dream2go.csv
102882 gr8h
rnn_train/gr8h.csv
102883 kishore
rnn_train/kishore.csv
102884 majay
rnn_train/

103076 MerelyMHP
rnn_train/MerelyMHP.csv
103077 akib9ctg
rnn_train/akib9ctg.csv
103078 avokadnok
rnn_train/avokadnok.csv
103079 erdomain
rnn_train/erdomain.csv
103081 AmeenAhmed
rnn_train/AmeenAhmed.csv
103082 Asad
rnn_train/Asad.csv
103083 ExCepTioNMaKeR
rnn_train/ExCepTioNMaKeR.csv
103084 Fuhrur
rnn_train/Fuhrur.csv
103085 horizon121
rnn_train/horizon121.csv
103087 nurakib.nstu
rnn_train/nurakib.nstu.csv
103088 velraman
rnn_train/velraman.csv
103089 Badgers_Prosen
rnn_train/Badgers_Prosen.csv
103091 sarasharaf75
rnn_train/sarasharaf75.csv
103092 Farnan.PUST
rnn_train/Farnan.PUST.csv
103093 Initiated
rnn_train/Initiated.csv
103094 Mohamed_Helmy
rnn_train/Mohamed_Helmy.csv
103095 aalim.kuet_14
rnn_train/aalim.kuet_14.csv
103096 inactive.coder
rnn_train/inactive.coder.csv
103099 worse
rnn_train/worse.csv
103100 forghani
rnn_train/forghani.csv
103101 futureview
rnn_train/futureview.csv
103102 greenNode
rnn_train/greenNode.csv
103103 medo_sherif
rnn_train/medo_sherif.csv
103104 Alfonze
rn

103295 eslam1077
rnn_train/eslam1077.csv
103296 Trasher
rnn_train/Trasher.csv
103297 Aaman007
rnn_train/Aaman007.csv
103298 AnhQuan
rnn_train/AnhQuan.csv
103299 NR_1995
rnn_train/NR_1995.csv
103301 dream557
rnn_train/dream557.csv
103303 messbah666
rnn_train/messbah666.csv
103304 sky1
rnn_train/sky1.csv
103305 ashwin_1995
rnn_train/ashwin_1995.csv
103306 faceoff
rnn_train/faceoff.csv
103307 kogy_92_2020
rnn_train/kogy_92_2020.csv
103308 haidmoham
rnn_train/haidmoham.csv
103309 kevinkowa
rnn_train/kevinkowa.csv
103310 Jongli_Coder
rnn_train/Jongli_Coder.csv
103311 alifanakgahoel63
rnn_train/alifanakgahoel63.csv
103312 prime_optimus
rnn_train/prime_optimus.csv
103313 task_in
rnn_train/task_in.csv
103314 Meyzdi_00
rnn_train/Meyzdi_00.csv
103315 Darth_Vader_
rnn_train/Darth_Vader_.csv
103316 Mitu_Bhowmick
rnn_train/Mitu_Bhowmick.csv
103317 shakib65
rnn_train/shakib65.csv
103318 abufarhad
rnn_train/abufarhad.csv
103320 hossaintanbir0
rnn_train/hossaintanbir0.csv
103321 safayet_chy
rnn_train/

103506 cobra
rnn_train/cobra.csv
103507 pipilika_mist
rnn_train/pipilika_mist.csv
103508 anand_prakash_yadav
rnn_train/anand_prakash_yadav.csv
103510 csaha505
rnn_train/csaha505.csv
103511 public00
rnn_train/public00.csv
103512 Ruman_Hossain
rnn_train/Ruman_Hossain.csv
103513 shabnam
rnn_train/shabnam.csv
103514 deepita
rnn_train/deepita.csv
103515 nikto351
rnn_train/nikto351.csv
103516 rashik1514
rnn_train/rashik1514.csv
103517 Moskaoud
rnn_train/Moskaoud.csv
103518 asha_42
rnn_train/asha_42.csv
103520 nayem_siddique_asif
rnn_train/nayem_siddique_asif.csv
103521 THton
rnn_train/THton.csv
103522 satya_jha
rnn_train/satya_jha.csv
103523 cat
rnn_train/cat.csv
103524 Ishraq_Nibir
rnn_train/Ishraq_Nibir.csv
103525 Kuyoku
rnn_train/Kuyoku.csv
103526 olives
rnn_train/olives.csv
103527 dgango
rnn_train/dgango.csv
103528 shubhangipriya
rnn_train/shubhangipriya.csv
103529 Tamtomus
rnn_train/Tamtomus.csv
103530 sherif1shrio
rnn_train/sherif1shrio.csv
103532 Challenge2017
rnn_train/Challenge2017.